# 测试工具


C语言的单元测试比较简单,可以使用一个叫[minunit](https://github.com/siu/minunit)的工具.它是专门针对C语言单元测试用的,没有侵入性.只有一个头文件.只要在下载下来就能直接用.

这个工具一般用于测试模块且单独使用,我们将compiler/C3的代码复制到test_tool下,并将minunit下载到test_tool/test/:

In [1]:
%%writefile ./code/test_tool/test/test.c
#include "minunit.h"
#include "binary_vector.h"

struct BINARY_VECTOR A = {1.0,2.0},B = {3.0,4.0};

MU_TEST(test_check_modA) {
    mu_check(VEC_mod(&A) == 2.236068);
}
MU_TEST(test_check_modB) {
    mu_check(VEC_mod(&B) == 5.000000);
}
MU_TEST(test_check_mulA_B) {
    mu_check(VEC_mul(&A,&B) == 11.000000);
}
MU_TEST(test_check_addA_B) {
    BINARY_VECTOR_P result = VEC_add(&A,&B);
    mu_check(result->x == 4.000000);
    mu_check(result->y == 6.000000);
}
MU_TEST_SUITE(test_suite) {
    MU_RUN_TEST(test_check_modA);
    MU_RUN_TEST(test_check_modB);
    MU_RUN_TEST(test_check_mulA_B);
    MU_RUN_TEST(test_check_addA_B);
}

int main(int argc, char *argv[]) {
    MU_RUN_SUITE(test_suite);
    MU_REPORT();
    return 0;
}


Overwriting ./code/test_tool/test/test.c


In [2]:
%%writefile ./code/test_tool/CMakeLists.txt
#项目编译环境
cmake_minimum_required (VERSION 2.8)
project (binary_vector)
include_directories(source/inc)
# 编译动态链接库
set(CMAKE_LIBRARY_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/build/lib)
aux_source_directory(${CMAKE_BINARY_DIR}/source/src DIR_LIB_SRCS)
add_library(vector SHARED ${DIR_LIB_SRCS})
install(TARGETS   vector
    LIBRARY DESTINATION  CMAKE_LIBRARY_OUTPUT_DIRECTORY  
)  
# 编译连接生成demo
set(CMAKE_RUNTIME_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/build/bin)
link_directories(${CMAKE_BINARY_DIR}/build/lib)
link_libraries(vector)
aux_source_directory(test DIR_SRCS)
add_executable(Demo_Test ${DIR_SRCS})

#测试
enable_testing()
add_test (TestRuns ${CMAKE_BINARY_DIR}/build/bin/Demo_Test)

Overwriting ./code/test_tool/CMakeLists.txt


In [3]:
!cmake ./code/test_tool/CMakeLists.txt

-- Configuring done
CMake Warning (dev):
  Policy CMP0042 is not set: MACOSX_RPATH is enabled by default.  Run "cmake
  --help-policy CMP0042" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

  MACOSX_RPATH is not specified for the following targets:

   vector

This warning is for project developers.  Use -Wno-dev to suppress it.

-- Generating done
-- Build files have been written to: /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/test_tool


In [4]:
!make -C ./code/test_tool/

[ 66%] Built target vector
Scanning dependencies of target Demo_Test
[ 83%] Building C object CMakeFiles/Demo_Test.dir/test/test.c.o
[100%] Linking C executable build/bin/Demo_Test
[100%] Built target Demo_Test


In [5]:
!make -C ./code/test_tool/ test

Running tests...
Test project /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/test_tool
    Start 1: TestRuns
1/1 Test #1: TestRuns .........................   Passed    0.01 sec

100% tests passed, 0 tests failed out of 1

Total Test time (real) =   0.01 sec


In [6]:
!./code/test_tool/build/bin/Demo_Test

F
test_check_modA failed:
	/Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/test_tool/test/test.c:7: VEC_mod(&A) == 2.236068
....

4 tests, 5 assertions, 1 failures

Finished in 0.00013472 seconds (real) 0.00006700 seconds (proc)



## Cmake的ctest工具

一般来说用Cmake都会用ctest作为测试工具,不过我并不推荐将它用在编写模块的时候,因为太重了,不过如果是用来测试一些比较复杂的命令行工具,他还是很不错的